In [114]:
import requests
from bs4 import BeautifulSoup
# import psycopg2
import itertools
import pandas as pd
import numpy as np

In [115]:
header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
}

In [116]:

df = pd.DataFrame()

#sayfa sayısı kadar döndürür (4.sayfaya kadar döndürdü)
for pageNumber in range(1):
    r = requests.get(f"https://www.kariyer.net/is-ilanlari/bilisim-{pageNumber}", headers=header)
    soup = BeautifulSoup(r.content, "lxml")
    ilanlar = soup.find_all("div", attrs={"class":"list-items"})

    for ilan in ilanlar:

        # Sayfadan ilan linkleri oluşturma
        url = ilan.a.get("href")
        link = "https://www.kariyer.net" + url

        #Yeni Link Çağırma
        r = requests.get(link, headers=header)
        soup = BeautifulSoup(r.content, "lxml")

        #Sayfadaki şirket ismi
        title = soup.find("a", attrs={"class": "company-name"}).text.strip()

        #Şirket niteliklerini alma
        labels = soup.find_all("div", attrs={"class": "label"})
        values = soup.find_all("div", attrs={"class": "value"})

        job_detail_content = soup.find("div", attrs="job-detail-content").text.replace("\xa0", "").replace("GENEL NİTELİKLER VE İŞ TANIMI", "").replace("QUALIFICATIONS AND JOB DESCRIPTION", "").strip()

        label_list, value_list = [], []

        #nitelikleri döndürür
        for label in labels:
            a = label.text.strip()
            label_list.append(a)

            for value in values:
                b = value.text.replace("başvuru","").replace("application", "").strip()
                value_list.append(b)
        
        zip_dict = dict(zip(label_list, value_list))

        zip_dict["Firma Adı"] = title

        zip_dict["Tanım"] = job_detail_content


        df_dictionary = pd.DataFrame([zip_dict])
        df = pd.concat([df, df_dictionary], ignore_index=True)



### Veri Ön İşleme

In [117]:
df.head(10)

,Çalışma Şekli,Pozisyon Seviyesi,Başvuru Sayısı,Departman,Firma Adı,Tanım,Tecrübe,Job Type,Position Level,Application Count,Department,Years of Experience
0,Tam Zamanlı,Uzman,91,Bilgi Teknolojileri / IT,Pratik Aday Servisi,"Danışmanlığını yürüttüğümüz,İmalat sektöründe ...",NaN,NaN,NaN,NaN,NaN,NaN
1,Tam Zamanlı,Uzman,57,Mühendislik,AltF4 Yazılım Reklam Danışmanlık,"Üniversitelerin Bilgisayar Mühendisliği , Bili...",NaN,NaN,NaN,NaN,NaN,NaN
2,Tam Zamanlı,NaN,81,Satış ve Pazarlama,Tanpera Teknoloji ve Endüstriyel Ürünler San. ...,"Isıtma, soğutma ve enerji sektörüne yönelik ek...",En az 2 yıl tecrübeli,NaN,NaN,NaN,NaN,NaN
3,Tam Zamanlı,NaN,388,Satış ve Pazarlama,REEDER,İstanbul Torium Avm Reeder Mağazamız bünyesind...,En az 1 yıl tecrübeli,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,SOFT AND SOLUTİON YAZILIM LİMİTED ŞİRKETİ,Job DescriptionSoft & Solution is a leading co...,NaN,Full-Time,Manager candidate,146,Information Technology,NaN
5,Tam Zamanlı,NaN,90,Bilgi Teknolojileri / IT,KARYA TEKNOLOJİ,Genel Nitelikler:• Üniversitelerin ilgili (Bil...,En az 3 yıl tecrübeli,NaN,NaN,NaN,NaN,NaN
6,Tam Zamanlı,Eleman,77,İş Geliştirme,Utarit Bilişim Hizmetleri Sanayi ve Ticaret .A.ş,MOBİL UYGULAMA GELİŞTİRİCİSİ ( ANDROID)- Andro...,NaN,NaN,NaN,NaN,NaN,NaN
7,Tam Zamanlı,Uzman,201,Operasyon,BESTERA YAZILIM DANIŞMANLIK SAN. TİC.LTDŞTİ,Yazılım Destek UzmanıEkibimize değer ve güç ka...,NaN,NaN,NaN,NaN,NaN,NaN
8,Tam Zamanlı,NaN,479,Satış ve Pazarlama,REEDER,Tekirdağ Tekira Avm Reeder Mağazamız bünyesind...,En az 1 yıl tecrübeli,NaN,NaN,NaN,NaN,NaN
9,Tam Zamanlı,Orta düzey yönetici,120,İdari İşler,HOLOCİTY ŞEHİR REKLAMCILIĞI VE TANITIM ANONİM ...,"Dünyada “ilk ve tek” olma özelliklerine sahip,...",NaN,NaN,NaN,NaN,NaN,NaN


In [119]:


df2 = df.copy()

#Departman birleştirme
combined_Department = np.where(df2["Departman"].isnull(), df2["Department"], df2["Departman"])
df2["Departman"] = combined_Department
df2.drop(columns=["Department"], inplace=True)


# Çalışma Şekli Birleştirme
combined_job = np.where(df2["Çalışma Şekli"].isnull(), df2["Job Type"], df2["Çalışma Şekli"])
df2["Çalışma Şekli"] = combined_job
df2.drop(columns=["Job Type"], inplace=True)
df2

#Pozisyon Seviyesi Birleştirme
position_Level = np.where(df2["Pozisyon Seviyesi"].isnull(), df2["Position Level"], df2["Pozisyon Seviyesi"])
df2["Pozisyon Seviyesi"] = position_Level
df2.drop(columns=["Position Level"], inplace=True)


#Başvuru Sayısı
app_count = np.where(df2["Başvuru Sayısı"].isnull(), df2["Application Count"], df2["Başvuru Sayısı"])
df2["Başvuru Sayısı"] = app_count
df2.drop(columns=["Application Count"], inplace=True)


#Tecrübe 
Experience = np.where(df2["Tecrübe"].isnull(), df2["Years of Experience"], df2["Tecrübe"])
df2["Tecrübe"] = Experience
df2.drop(columns=["Years of Experience"], inplace=True)
df2

df2 = df2[['Firma Adı', 'Departman', 'Çalışma Şekli', 'Pozisyon Seviyesi', 'Tecrübe', "Başvuru Sayısı", 'Tanım']]

df = df2


In [120]:
df.head(10)

,Firma Adı,Departman,Çalışma Şekli,Pozisyon Seviyesi,Tecrübe,Başvuru Sayısı,Tanım
0,Pratik Aday Servisi,Bilgi Teknolojileri / IT,Tam Zamanlı,Uzman,NaN,91,"Danışmanlığını yürüttüğümüz,İmalat sektöründe ..."
1,AltF4 Yazılım Reklam Danışmanlık,Mühendislik,Tam Zamanlı,Uzman,NaN,57,"Üniversitelerin Bilgisayar Mühendisliği , Bili..."
2,Tanpera Teknoloji ve Endüstriyel Ürünler San. ...,Satış ve Pazarlama,Tam Zamanlı,NaN,En az 2 yıl tecrübeli,81,"Isıtma, soğutma ve enerji sektörüne yönelik ek..."
3,REEDER,Satış ve Pazarlama,Tam Zamanlı,NaN,En az 1 yıl tecrübeli,388,İstanbul Torium Avm Reeder Mağazamız bünyesind...
4,SOFT AND SOLUTİON YAZILIM LİMİTED ŞİRKETİ,Information Technology,Full-Time,Manager candidate,NaN,146,Job DescriptionSoft & Solution is a leading co...
5,KARYA TEKNOLOJİ,Bilgi Teknolojileri / IT,Tam Zamanlı,NaN,En az 3 yıl tecrübeli,90,Genel Nitelikler:• Üniversitelerin ilgili (Bil...
6,Utarit Bilişim Hizmetleri Sanayi ve Ticaret .A.ş,İş Geliştirme,Tam Zamanlı,Eleman,NaN,77,MOBİL UYGULAMA GELİŞTİRİCİSİ ( ANDROID)- Andro...
7,BESTERA YAZILIM DANIŞMANLIK SAN. TİC.LTDŞTİ,Operasyon,Tam Zamanlı,Uzman,NaN,201,Yazılım Destek UzmanıEkibimize değer ve güç ka...
8,REEDER,Satış ve Pazarlama,Tam Zamanlı,NaN,En az 1 yıl tecrübeli,479,Tekirdağ Tekira Avm Reeder Mağazamız bünyesind...
9,HOLOCİTY ŞEHİR REKLAMCILIĞI VE TANITIM ANONİM ...,İdari İşler,Tam Zamanlı,Orta düzey yönetici,NaN,120,"Dünyada “ilk ve tek” olma özelliklerine sahip,..."


In [121]:
df.shape

(52, 7)

In [122]:
df["Tanım"][4]

"Job DescriptionSoft & Solution is a leading company in Albania, with several years of experience in the field of Information and Telecommunication Technology, which provides services in the field of information technology, develops applications of all categories and profiles.We are expanding our team in Istanbul, Turkey with 50 new members who will be engaged in international projects.Our portfolio includes:Development of dedicated programsIntegration and implementation of systemsMaintenance of systemsTrainingsThe company’s mission is to provide maximum quality for each service, based on the technology and opportunities it provides. With strong confidence in the values we offer, we work alongside our customers to design and build contemporary solutions, based on the flexible platforms we have created.Soft & Solution is seeking .Net Developers to join our team in Istanbul. This individual will work with the Development Team Leader and will be tasked with completing assigned features on

In [ ]:
# excel tablosu oluşturur

file_name = 'MarksData.xlsx'

df.to_excel(file_name)